# Drake Scene with two UR5e robots

To run this notebook, you need access to the currently private `airo-planner` and install it.

In [ ]:
import os
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from pydrake.multibody.plant import DiscreteContactSolver

# from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# config = VisualizationConfig(publish_contacts=True, enable_alpha_sliders=True)
# ApplyVisualizationConfig(config, builder=builder, plant=plant, meshcat=meshcat)

# This get rid ot the warning for the mimic joints in the Robotiq gripper
plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
cube_urdf = "cube_and_cylinder.urdf"
table_urdf = "table.urdf"

arm_left_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
arm_right_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_right")
gripper_left_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper_left")
gripper_right_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper_right")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_left_frame = plant.GetFrameByName("base_link", arm_left_index)
arm_right_frame = plant.GetFrameByName("base_link", arm_right_index)
arm_left_wrist_frame = plant.GetFrameByName("wrist_3_link", arm_left_index)
arm_right_wrist_frame = plant.GetFrameByName("wrist_3_link", arm_right_index)
gripper_left_frame = plant.GetFrameByName("base_link", gripper_left_index)
gripper_right_frame = plant.GetFrameByName("base_link", gripper_right_index)
table_frame = plant.GetFrameByName("base_link", table_index)

distance_between_arms = 0.9
distance_between_arms_half = distance_between_arms / 2

plant.WeldFrames(world_frame, arm_left_frame)
plant.WeldFrames(world_frame, arm_right_frame, RigidTransform([distance_between_arms, 0, 0]))
plant.WeldFrames(arm_left_wrist_frame, gripper_left_frame)
plant.WeldFrames(arm_right_wrist_frame, gripper_right_frame)
plant.WeldFrames(world_frame, table_frame, RigidTransform([distance_between_arms_half, 0, 0]))


# plant.Finalize()

# Set up collision checking
diagram = robot_diagram_builder.Build()


# Not sure if this is needed
def _configuration_distance(q1, q2):
    return np.linalg.norm(q1 - q2)


collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_left_index, arm_right_index],
    configuration_distance_function=_configuration_distance,
    edge_step_size=0.125,
)

# Create default contexts ~= state
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

In [ ]:
q = plant.GetPositions(plant_context, arm_left_index).tolist()
q

In [ ]:
q_all = plant.GetPositions(plant_context)
collision_checker.CheckConfigCollisionFree(q_all)

In [ ]:
q_new = q.copy()
q_new[1] = -np.pi / 2
plant.SetPositions(plant_context, arm_left_index, q_new)
plant.SetPositions(plant_context, arm_right_index, q_new)

diagram.ForcedPublish(context)

q_all = plant.GetPositions(plant_context)
collision_checker.CheckConfigCollisionFree(q_all)

In [ ]:
collision_checker.CheckConfigCollisionFree(np.zeros(12))